# Face Recognition Accuracy Testing with LFW Dataset

This notebook evaluates the accuracy of your face recognition system using the Labeled Faces in the Wild (LFW) dataset.

Ensure the following prerequisites are met:
- The `detector.py` and `model.py` files are functional and accessible.
- The required dependencies are installed, including `sklearn` and `numpy`.

In [3]:
# Import các thư viện cần thiết
import os
import cv2
import numpy as np
import torch
from torchvision import transforms
from scrfd.detector import SCRFD
from arcface.model import iresnet_inference
from arcface.utils import compare_encodings, read_features
from face_alignment.alignment import norm_crop
import matplotlib.pyplot as plt
from PIL import Image

# Thiết lập thiết bị (GPU nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Tải mô hình phát hiện khuôn mặt SCRFD
detector = SCRFD(model_file="scrfd/weights/scrfd_2.5g_bnkps.onnx")
detector.prepare(ctx_id=0 if torch.cuda.is_available() else -1)

# Tải mô hình nhận diện ArcFace
recognizer = iresnet_inference(
    model_name="r100", path="arcface/weights/arcface_r100.pth", device=device
)

# Tải file features.npz
features_path = "./datasets/face_features/feature.npz"
images_names, images_embs = read_features(features_path)

print(f"Loaded {len(images_names)} face embeddings from {features_path}")


TypeError: cannot unpack non-iterable NoneType object

In [2]:
def read_features(feature_path):
    """
    Đọc các vector đặc trưng và nhãn từ file features.npz.
    """
    if not os.path.exists(feature_path):
        print(f"File {feature_path} không tồn tại.")
        return None

    try:
        data = np.load(feature_path, allow_pickle=True)
        if "images_names" in data and "images_embs" in data:
            return data["images_names"], data["images_embs"]
        else:
            print("File không chứa đủ dữ liệu cần thiết (images_names hoặc images_embs).")
            return None
    except Exception as e:
        print(f"Lỗi khi đọc file features.npz: {e}")
        return None


In [4]:
import numpy as np
import os

features_path = "./datasets/face_features/feature.npz"

# Kiểm tra file tồn tại
if not os.path.exists(features_path):
    print(f"File {features_path} không tồn tại.")
else:
    print(f"File {features_path} tồn tại.")

    # Thử mở file và kiểm tra nội dung
    try:
        data = np.load(features_path, allow_pickle=True)
        print("Các trường trong file features.npz:", data.files)
        
        if "images_names" in data and "images_embs" in data:
            print("File chứa đầy đủ dữ liệu.")
            images_names = data["images_names"]
            images_embs = data["images_embs"]
            print(f"Đọc được {len(images_names)} nhãn và {images_embs.shape[0]} vector đặc trưng.")
        else:
            print("File không chứa đủ dữ liệu cần thiết (images_names hoặc images_embs).")
    except Exception as e:
        print(f"Lỗi khi đọc file: {e}")


File ./datasets/face_features/feature.npz tồn tại.
Các trường trong file features.npz: ['images_name', 'images_emb']
File không chứa đủ dữ liệu cần thiết (images_names hoặc images_embs).


In [5]:
def read_features(feature_path):
    """
    Đọc các vector đặc trưng và nhãn từ file features.npz.
    """
    if not os.path.exists(feature_path):
        print(f"File {feature_path} không tồn tại.")
        return None

    try:
        data = np.load(feature_path, allow_pickle=True)
        # Sử dụng tên trường thực tế trong file
        if "images_name" in data and "images_emb" in data:
            return data["images_name"], data["images_emb"]
        else:
            print("File không chứa đủ dữ liệu cần thiết (images_name hoặc images_emb).")
            return None
    except Exception as e:
        print(f"Lỗi khi đọc file features.npz: {e}")
        return None


In [6]:
data = np.load("./datasets/face_features/feature.npz", allow_pickle=True)
print("Các trường trong file features.npz:", data.files)

# Kiểm tra kích thước và dữ liệu của các trường
if "images_name" in data and "images_emb" in data:
    images_names = data["images_name"]
    images_embs = data["images_emb"]
    print(f"Đọc được {len(images_names)} nhãn và {images_embs.shape[0]} vector đặc trưng.")
else:
    print("File không chứa đủ dữ liệu cần thiết.")


Các trường trong file features.npz: ['images_name', 'images_emb']
Đọc được 707 nhãn và 707 vector đặc trưng.


In [7]:
import numpy as np
from collections import defaultdict

def group_features_by_name(features_path, output_path):
    """
    Gộp các vector đặc trưng theo nhãn (group by name) và lưu file mới.
    
    Args:
        features_path (str): Đường dẫn tới file features.npz.
        output_path (str): Đường dẫn để lưu file features.npz sau khi gộp.
    """
    # Đọc file features.npz
    data = np.load(features_path, allow_pickle=True)
    images_names = data["images_name"]  # Nhãn của các vector
    images_embs = data["images_emb"]    # Vector đặc trưng

    # Tạo dictionary để nhóm vector theo nhãn
    grouped_embeddings = defaultdict(list)

    # Nhóm vector đặc trưng theo nhãn
    for name, emb in zip(images_names, images_embs):
        grouped_embeddings[name].append(emb)

    # Gộp vector (tính trung bình)
    merged_names = []
    merged_embs = []

    for name, embs in grouped_embeddings.items():
        merged_names.append(name)
        merged_embs.append(np.mean(embs, axis=0))  # Tính trung bình vector đặc trưng

    # Chuyển sang định dạng numpy array
    merged_names = np.array(merged_names)
    merged_embs = np.array(merged_embs)

    # Lưu file mới
    np.savez_compressed(output_path, images_name=merged_names, images_emb=merged_embs)
    print(f"Đã gộp và lưu file mới tại {output_path}")
    print(f"Số lượng nhãn sau khi gộp: {len(merged_names)}")


In [8]:
features_path = "./datasets/face_features/feature.npz"
output_path = "./datasets/face_features/merged_feature.npz"

group_features_by_name(features_path, output_path)


Đã gộp và lưu file mới tại ./datasets/face_features/merged_feature.npz
Số lượng nhãn sau khi gộp: 37


In [15]:
# Đọc file đã gộp
data = np.load(output_path, allow_pickle=True)
merged_names = data["images_name"]
merged_embs = data["images_emb"]

print(f"Các nhãn trong file mới: {merged_names}")
print(f"Số lượng nhãn sau khi gộp: {len(merged_names)}")



Các nhãn trong file mới: ['person_10_tom_holland' 'person_10_tom_holland' 'person_10_tom_holland'
 'person_10_tom_holland' 'person_10_tom_holland' 'person_10_tom_holland'
 'person_10_tom_holland' 'person_10_tom_holland' 'person_10_tom_holland'
 'person_10_tom_holland' 'person_10_tom_holland' 'person_10_tom_holland'
 'person_10_tom_holland' 'person_10_tom_holland' 'person_10_tom_holland'
 'person_10_tom_holland' 'person_10_tom_holland' 'person_11_timmothy'
 'person_11_timmothy' 'person_11_timmothy' 'person_11_timmothy'
 'person_11_timmothy' 'person_11_timmothy' 'person_11_timmothy'
 'person_11_timmothy' 'person_11_timmothy' 'person_11_timmothy'
 'person_11_timmothy' 'person_12_hiddleston' 'person_12_hiddleston'
 'person_12_hiddleston' 'person_12_hiddleston' 'person_12_hiddleston'
 'person_12_hiddleston' 'person_12_hiddleston' 'person_12_hiddleston'
 'person_12_hiddleston' 'person_12_hiddleston' 'person_13_katty_perry'
 'person_13_katty_perry' 'person_13_katty_perry' 'person_13_katty_per

In [16]:
import numpy as np

def normalize_labels(features_path, output_path):
    """
    Chuẩn hóa nhãn trong file features.npz bằng cách thay khoảng trắng bằng dấu gạch dưới.
    
    Args:
        features_path (str): Đường dẫn tới file features.npz cần chuẩn hóa.
        output_path (str): Đường dẫn để lưu file đã chuẩn hóa.
    """
    # Đọc file features.npz
    data = np.load(features_path, allow_pickle=True)
    images_names = data["images_name"]
    images_embs = data["images_emb"]

    # Chuẩn hóa nhãn
    normalized_names = [name.replace(" ", "_") for name in images_names]

    # Lưu lại file mới với nhãn đã chuẩn hóa
    np.savez_compressed(output_path, images_name=np.array(normalized_names), images_emb=np.array(images_embs))
    print(f"Đã chuẩn hóa nhãn và lưu file mới tại {output_path}")


In [17]:
features_path = "./datasets/face_features/feature.npz"
output_path = "./datasets/face_features/normalized_feature.npz"


In [18]:
normalize_labels(features_path, output_path)


Đã chuẩn hóa nhãn và lưu file mới tại ./datasets/face_features/normalized_feature.npz


In [19]:
import os
import numpy as np
import cv2
import torch
from torchvision import transforms
from arcface.model import iresnet_inference
from arcface.utils import compare_encodings
import matplotlib.pyplot as plt

# Thiết lập thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tải mô hình nhận diện ArcFace
recognizer = iresnet_inference(
    model_name="r100", path="arcface/weights/arcface_r100.pth", device=device
)

# Đường dẫn tới file features.npz đã được chuẩn hóa
features_path = "./datasets/face_features/normalized_feature.npz"

In [20]:
# Đọc file features.npz
data = np.load(features_path, allow_pickle=True)
images_names = data["images_name"]
images_embs = data["images_emb"]

print(f"Đã tải {len(images_names)} nhãn và vector đặc trưng từ {features_path}.")


Đã tải 707 nhãn và vector đặc trưng từ ./datasets/face_features/normalized_feature.npz.


In [21]:
features_path = "./datasets/face_features/normalized_feature.npz"
output_path = "./datasets/face_features/merged_feature.npz"

group_features_by_name(features_path, output_path)

Đã gộp và lưu file mới tại ./datasets/face_features/merged_feature.npz
Số lượng nhãn sau khi gộp: 37


In [22]:
# Hàm trích xuất vector đặc trưng từ ảnh khuôn mặt
@torch.no_grad()
def get_feature(face_image):
    """
    Trích xuất vector đặc trưng từ ảnh khuôn mặt đã được phát hiện.
    """
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((112, 112)),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])
    face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
    face_tensor = preprocess(face_image).unsqueeze(0).to(device)

    emb = recognizer(face_tensor).cpu().numpy()
    return emb / np.linalg.norm(emb)

In [30]:
def recognition(face_image):
    """
    Nhận diện khuôn mặt từ ảnh khuôn mặt.
    """
    query_emb = get_feature(face_image)
    score, idx = compare_encodings(query_emb, images_embs)
    recognized_name = images_names[idx]
    return score[0], recognized_name


In [ ]:
def show_image_with_results(image_path, true_name):
    """
    Hiển thị kết quả nhận diện từ ảnh đầu vào.
    
    Args:
        image_path (str): Đường dẫn tới ảnh.
        true_name (str): Tên thực tế của người trong ảnh.
    """
    # Đọc ảnh
    face_image = cv2.imread(image_path)
    if face_image is None:
        print(f"Không thể đọc ảnh từ {image_path}")
        return

    # Nhận diện khuôn mặt
    score, recognized_name = recognition(face_image)

    # Hiển thị ảnh và kết quả
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"True: {true_name}\nPredicted: {recognized_name} ({score:.2f})")
    plt.show()

    # In kết quả chi tiết
    print(f"Kết quả nhận diện từ {image_path}:")
    print(f"  - Tên nhận diện: {recognized_name}")
    print(f"  - Điểm số: {score:.2f}")
    print(f"  - Tên thực tế: {true_name}")

In [26]:
# Đánh giá trên một ảnh
def evaluate_single_image(image_path, true_name):
    """
    Đánh giá kết quả nhận diện trên một ảnh.
    
    Args:
        image_path (str): Đường dẫn tới ảnh đã cắt khuôn mặt.
        true_name (str): Tên thực tế của người trong ảnh.

    Returns:
        bool: True nếu nhận diện đúng, False nếu sai.
    """
    face_image = cv2.imread(image_path)
    if face_image is None:
        print(f"Không thể đọc ảnh: {image_path}")
        return False

    score, recognized_name = recognition(face_image)
    print(f"Kết quả từ {image_path}:")
    print(f"  - Dự đoán: {recognized_name} (Score: {score:.2f})")
    print(f"  - Thực tế: {true_name}")

    return recognized_name == true_name

In [27]:
# Đánh giá toàn bộ dataset
def evaluate_model(dataset_dir):
    """
    Đánh giá mô hình trên toàn bộ tập dữ liệu đã detect.
    
    Args:
        dataset_dir (str): Đường dẫn tới thư mục chứa dataset.

    Returns:
        accuracy (float): Độ chính xác của mô hình.
    """
    total_images = 0
    correct_recognitions = 0
    false_recognitions = 0

    for person_name in os.listdir(dataset_dir):
        person_dir = os.path.join(dataset_dir, person_name)
        if not os.path.isdir(person_dir):
            continue

        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            total_images += 1
            if evaluate_single_image(img_path, person_name):
                correct_recognitions += 1
            else:
                false_recognitions += 1

    accuracy = correct_recognitions / total_images * 100
    print(f"Độ chính xác của mô hình: {accuracy:.2f}%")
    print(f"Tổng số ảnh: {total_images}")
    print(f"Số nhận diện đúng: {correct_recognitions}")
    print(f"Số nhận diện sai: {false_recognitions}")
    return accuracy


In [31]:
# Hiển thị kết quả cho một ảnh cụ thể
image_path = "./datasets/data/person_10_tom_holland/1.jpg"
true_name = "person_10_tom_holland"
show_image_with_results(image_path, true_name)

# Đánh giá toàn bộ dataset
dataset_dir = "./datasets/data"
evaluate_model(dataset_dir)

Kết quả từ ./datasets/data\person_10_tom_holland\0.jpg:
  - Dự đoán: person_10_tom_holland (Score: 1.00)
  - Thực tế: person_10_tom_holland
Kết quả từ ./datasets/data\person_10_tom_holland\1.jpg:
  - Dự đoán: person_10_tom_holland (Score: 1.00)
  - Thực tế: person_10_tom_holland
Kết quả từ ./datasets/data\person_10_tom_holland\10.jpg:
  - Dự đoán: person_10_tom_holland (Score: 1.00)
  - Thực tế: person_10_tom_holland
Kết quả từ ./datasets/data\person_10_tom_holland\11.jpg:
  - Dự đoán: person_10_tom_holland (Score: 1.00)
  - Thực tế: person_10_tom_holland
Kết quả từ ./datasets/data\person_10_tom_holland\12.jpg:
  - Dự đoán: person_10_tom_holland (Score: 1.00)
  - Thực tế: person_10_tom_holland
Kết quả từ ./datasets/data\person_10_tom_holland\13.jpg:
  - Dự đoán: person_10_tom_holland (Score: 1.00)
  - Thực tế: person_10_tom_holland
Kết quả từ ./datasets/data\person_10_tom_holland\14.jpg:
  - Dự đoán: person_10_tom_holland (Score: 1.00)
  - Thực tế: person_10_tom_holland
Kết quả từ ./da

92.31692677070828

In [32]:
#shuffle dữ liệu 
import random

def evaluate_model(dataset_dir, shuffle=True):
    """
    Đánh giá mô hình trên toàn bộ tập dữ liệu đã detect và xáo trộn dữ liệu nếu cần.
    
    Args:
        dataset_dir (str): Đường dẫn tới thư mục chứa dataset.
        shuffle (bool): Có xáo trộn dữ liệu trước khi dự đoán không.

    Returns:
        accuracy (float): Độ chính xác của mô hình.
    """
    total_images = 0
    correct_recognitions = 0
    false_recognitions = 0

    # Tạo danh sách tất cả ảnh và nhãn
    all_images = []
    for person_name in os.listdir(dataset_dir):
        person_dir = os.path.join(dataset_dir, person_name)
        if not os.path.isdir(person_dir):
            continue

        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            all_images.append((img_path, person_name))

    # Shuffle dữ liệu
    if shuffle:
        random.shuffle(all_images)

    # Dự đoán trên từng ảnh
    for img_path, true_name in all_images:
        total_images += 1
        if evaluate_single_image(img_path, true_name):
            correct_recognitions += 1
        else:
            false_recognitions += 1

    accuracy = correct_recognitions / total_images * 100
    print(f"Độ chính xác của mô hình: {accuracy:.2f}%")
    print(f"Tổng số ảnh: {total_images}")
    print(f"Số nhận diện đúng: {correct_recognitions}")
    print(f"Số nhận diện sai: {false_recognitions}")
    return accuracy


In [33]:
def evaluate_single_image(image_path, true_name):
    """
    Đánh giá kết quả nhận diện trên một ảnh.
    
    Args:
        image_path (str): Đường dẫn tới ảnh đã cắt khuôn mặt.
        true_name (str): Tên thực tế của người trong ảnh.

    Returns:
        bool: True nếu nhận diện đúng, False nếu sai.
    """
    face_image = cv2.imread(image_path)
    if face_image is None:
        print(f"Không thể đọc ảnh: {image_path}")
        return False

    score, recognized_name = recognition(face_image)
    print(f"Kết quả từ {image_path}:")
    print(f"  - Dự đoán: {recognized_name} (Score: {score:.2f})")
    print(f"  - Thực tế: {true_name}")

    return recognized_name == true_name


In [34]:
dataset_dir = "./datasets/data"
evaluate_model(dataset_dir, shuffle=True)

Kết quả từ ./datasets/data\person_43_dang\19.jpg:
  - Dự đoán: person_43_dang (Score: 1.00)
  - Thực tế: person_43_dang
Kết quả từ ./datasets/data\person_11_timmothy\20.jpg:
  - Dự đoán: person_11_timmothy (Score: 1.00)
  - Thực tế: person_11_timmothy
Kết quả từ ./datasets/data\person_11_timmothy\2.jpg:
  - Dự đoán: person_11_timmothy (Score: 1.00)
  - Thực tế: person_11_timmothy
Kết quả từ ./datasets/data\person_39_crisdevil\13.jpg:
  - Dự đoán: person_39_crisdevil (Score: 1.00)
  - Thực tế: person_39_crisdevil
Kết quả từ ./datasets/data\person_36_brad\14.jpg:
  - Dự đoán: person_36_brad (Score: 1.00)
  - Thực tế: person_36_brad
Kết quả từ ./datasets/data\person_11_timmothy\12.jpg:
  - Dự đoán: person_11_timmothy (Score: 1.00)
  - Thực tế: person_11_timmothy
Kết quả từ ./datasets/data\person_37_hurrykhng\10.jpg:
  - Dự đoán: person_37_hurrykhng (Score: 1.00)
  - Thực tế: person_37_hurrykhng
Kết quả từ ./datasets/data\person_27_mck\3.jpg:
  - Dự đoán: person_27_mck (Score: 1.00)
  - Th

92.31692677070828

In [35]:
def show_random_results(dataset_dir, num_images=5):
    """
    Hiển thị kết quả nhận diện ngẫu nhiên từ dataset.
    
    Args:
        dataset_dir (str): Đường dẫn tới thư mục dataset.
        num_images (int): Số ảnh ngẫu nhiên để hiển thị.
    """
    all_images = []
    for person_name in os.listdir(dataset_dir):
        person_dir = os.path.join(dataset_dir, person_name)
        if not os.path.isdir(person_dir):
            continue

        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            all_images.append((img_path, person_name))

    # Shuffle và lấy num_images ảnh
    random.shuffle(all_images)
    sample_images = all_images[:num_images]

    # Hiển thị từng ảnh
    for img_path, true_name in sample_images:
        show_image_with_results(img_path, true_name)


In [36]:
show_random_results(dataset_dir, num_images=5)
